In [1]:
import numpy as np
import librosa
import os
import glob
import gc
import tensorflow as tf
from tensorflow import keras
import json

# from google.colab import drive
# drive.mount('/content/gdrive')
driveLoc='../'

In [2]:
training_samples = 4000
validation_samples = 400
ratio_of_bonafide_samples = 0.5
data_iterations = 10
epochs_per_iteration = 5
size_of_batch = 128
alpha = 0.001 #learning_rate

In [3]:
bonafide_label = 0 #not-fake

desired_sampling_rate = 16000 # in Hz
time_duration = 4 # in seconds

window_size = 108 #in ms
window_shift = 10 #in ms
pointFFT = 1728

data_loc = driveLoc + 'CQCC/'

samples_per_folder = 500

window_size = int(window_size*desired_sampling_rate/1000) # converting to samples
window_shift = int(window_shift*desired_sampling_rate/1000) # converting to samples

In [4]:
training_data_mean = np.loadtxt(data_loc+'train_mean_CQCC.txt', delimiter=',')
training_data_stddev = np.loadtxt(data_loc+'train_stddev_CQCC.txt', delimiter=',')
(dim1, dim2) = np.shape(training_data_mean)
print(dim1, dim2)

def GenerateFeatures(filename): #Generates Normalized Features
    features = np.loadtxt(filename, delimiter=',')
    return (features-training_data_mean)/training_data_stddev

90 469


In [5]:
with open(data_loc + "bonafide_test_set_files.txt", "r") as fp:
    bonafide_test_files = json.load(fp)

with open(data_loc + "spoof_test_set_files.txt", "r") as fp:
    spoof_test_files = json.load(fp)

In [6]:
#Checknig spoof and bonafide files
from collections import Counter
a, b = len(Counter(spoof_test_files).keys()),len(Counter(bonafide_test_files).keys())
print(a,b)


4400 500


In [7]:
print('..'+ bonafide_test_files[0][37:])
print(len('/content/gdrive/My Drive/MLSP Project'))

../CQCC/train/bonafide_4/LA_T_2170687.txt
37


# Perofrmance on Test data

In [8]:
# Testing data 
from sklearn.metrics import accuracy_score,f1_score


results = [] # for each model accuracy and f1 score
models=[]
for model_name in os.listdir(driveLoc+'/Models3'):
    print(model_name)
    models.append(keras.models.load_model(driveLoc+'/Models3/' + model_name+'/13'))
    results.append([0,0])
    
    
y_pred = np.zeros((len(models),len(spoof_test_files) + len(bonafide_test_files))) # depends on model index
y_true=np.append(np.ones(len(spoof_test_files)),np.zeros(len(bonafide_test_files)))
sample_no = 0
for filename in spoof_test_files:
    a = GenerateFeatures('..'+ filename[37:])
    if np.isnan(a).any() or np.isinf(a).any():
        for i in range(len(models)):
            y_pred[i,sample_no] = 1
    else:
        dim1,dim2 = a.shape
        a=a.reshape((1,dim1,dim2,1))
        for i in range(len(models)):
            y_pred[i,sample_no]=(np.argmax(models[i].predict(a)))
    sample_no+=1
        
for filename in bonafide_test_files:
    a = GenerateFeatures('..'+ filename[37:])
    dim1,dim2 = a.shape
    a=a.reshape((1,dim1,dim2,1))
    for i in range(len(models)):
        y_pred[i,sample_no]=(np.argmax(models[i].predict(a)))
    sample_no+=1

np.savetxt('prediction_for_models.txt',y_pred)




21_june03
21_june01
21_june07
21_june05
21_june04
21_june02
21_june10
21_june08
21_june09
21_june06


# Accuracy and F1 score 

In [9]:
for i in range(len(models)):
    results[i][1] = f1_score(y_true, y_pred[i])
    results[i][0] = accuracy_score(y_true, y_pred[i])

print(results)

[[0.9993877551020408, 0.9996592070884925], [0.9979591836734694, 0.9988633780404638], [0.9997959183673469, 0.9998863765481194], [0.9981632653061224, 0.998976691301876], [0.9989795918367347, 0.9994317536083646], [0.9991836734693877, 0.9995455578277663], [0.9993877551020408, 0.999658974650449], [1.0, 1.0], [0.9995918367346939, 0.9997727272727273], [0.9993877551020408, 0.9996590521650188]]


# Performance on Eval Data


In [15]:
model = keras.models.load_model(driveLoc+'/Models3/21_june04/13')
model.summary()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_169 (Conv2D)          (None, 45, 235, 4)        40        
_________________________________________________________________
batch_normalization_191 (Bat (None, 45, 235, 4)        16        
_________________________________________________________________
conv2d_170 (Conv2D)          (None, 42, 232, 4)        260       
_________________________________________________________________
batch_normalization_192 (Bat (None, 42, 232, 4)        16        
_________________________________________________________________
conv2d_171 (Conv2D)          (None, 40, 230, 4)        148       
_________________________________________________________________
batch_normalization_193 (Bat (None, 40, 230, 4)        16        
_________________________________________________________________
max_pooling2d_72 (MaxPooling (None, 20, 115, 4)      

In [11]:
# Eval data :
from sklearn.metrics import accuracy_score,f1_score


results = [] # for each model accuracy and f1 score
y_true=[]
y_pred=[]
models=[]
for model_name in os.listdir(driveLoc+'/Models3'):
    print(model_name)
    models.append(keras.models.load_model(driveLoc+'/Models3/' + model_name+'/13'))
    results.append([0,0])
    y_pred.append([])

    
    



COUNT = 0
for foldername in os.listdir(driveLoc+'/CQCC/eval'):
    COUNT += 1
    print(COUNT, foldername)
    for filename in os.listdir(driveLoc+'/CQCC/eval/'+foldername):
        try:
            a = GenerateFeatures(driveLoc + '/CQCC/eval/'+foldername+'/'+filename)
        except:
            print(foldername, filename)
            continue
        if 'bonafide' in foldername : 
            y_true.append(0)
        else: y_true.append(1)

        if np.isnan(a).any() or np.isinf(a).any():
            for i in range(len(models)):
                y_pred[i].append('spoof' in foldername)
        else:
            dim1,dim2 = a.shape
            a=a.reshape((1,dim1,dim2,1))
            for i in range(len(models)):
                y_pred[i].append(np.argmax(models[i].predict(a)))
        

np.savetxt('eval_predictions_CQCC2.txt',y_pred)
np.savetxt('eval_label_CQCC2.txt',y_true)


21_june03
21_june01
21_june07
21_june05
21_june04
21_june02
21_june10
21_june08
21_june09
21_june06
1 spoof_120
2 spoof_68
3 spoof_21
4 spoof_42
5 spoof_51
6 spoof_87
7 spoof_28
8 spoof_111
9 spoof_47
10 spoof_34
11 spoof_91
12 spoof_22
13 spoof_95
14 spoof_39
15 spoof_59
16 spoof_63
17 spoof_56
18 spoof_84
19 spoof_117
20 spoof_53
21 spoof_36
22 spoof_67
23 spoof_123
24 spoof_112
25 spoof_82
26 spoof_1
27 spoof_106
28 spoof_121
29 spoof_8
30 spoof_9
31 spoof_27
32 spoof_19
33 spoof_35
34 spoof_41
35 spoof_99
36 spoof_55
37 spoof_17
38 spoof_48
39 bonafide_8
40 spoof_66
41 spoof_75
42 bonafide_1
43 spoof_43
44 spoof_113
45 bonafide_0
46 spoof_89
47 spoof_73
48 spoof_115
49 spoof_118
50 spoof_74
51 spoof_25
52 spoof_5
53 spoof_16
54 spoof_61
55 spoof_101
56 spoof_46
57 spoof_3
58 spoof_76
59 spoof_50
60 spoof_33
61 spoof_110
62 spoof_40
63 spoof_94
64 spoof_23
65 bonafide_10
66 spoof_85
67 spoof_103
68 spoof_13
69 spoof_4
70 spoof_72
71 spoof_58
72 bonafide_12
73 spoof_14
74 spoof_122
7

In [12]:
for i in range(len(models)):
    results[i][1] = f1_score(y_true, y_pred[i])
    results[i][0] = accuracy_score(y_true, y_pred[i])

print(results)

[[0.6279180762805845, 0.738656307310051], [0.6426435700548873, 0.7514523105161928], [0.6581692098207392, 0.7646586966396381], [0.6629279728231116, 0.7688130632365401], [0.701573620450047, 0.8005479092196984], [0.6515715148027008, 0.7590732166603573], [0.6356107079186378, 0.745260058881256], [0.6333646840827099, 0.7432918558707319], [0.6357651220573578, 0.7453105215112342], [0.6366354562937799, 0.7461483392011298]]


In [13]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 22:14:06


In [14]:
for j in range(len(models)):
    countBonafide = 0
    countSpoof = 0
    countBonafideClassifiedSpoof = 0
    countSpoofClassifiedBonafide = 0
    for i in range(len(y_true)):
        if(y_true[i] > 0.5):
            countSpoof += 1
            if(y_pred[j][i] == 0):
                countSpoofClassifiedBonafide += 1
        elif(y_true[i] < 0.5):
            countBonafide += 1
            if(y_pred[j][i] == 1):
                countBonafideClassifiedSpoof += 1
    print("Model {}...".format(j))
    print("Bonafide Samples = {}, Incorrectly classified = {}".format(countBonafide, countBonafideClassifiedSpoof))
    print("Spoof Samples = {}, Incorrectly classified = {}".format(countSpoof, countSpoofClassifiedBonafide))

Model 0...
Bonafide Samples = 7355, Incorrectly classified = 82
Spoof Samples = 63882, Incorrectly classified = 26424
Model 1...
Bonafide Samples = 7355, Incorrectly classified = 58
Spoof Samples = 63882, Incorrectly classified = 25399
Model 2...
Bonafide Samples = 7355, Incorrectly classified = 29
Spoof Samples = 63882, Incorrectly classified = 24322
Model 3...
Bonafide Samples = 7355, Incorrectly classified = 56
Spoof Samples = 63882, Incorrectly classified = 23956
Model 4...
Bonafide Samples = 7355, Incorrectly classified = 41
Spoof Samples = 63882, Incorrectly classified = 21218
Model 5...
Bonafide Samples = 7355, Incorrectly classified = 40
Spoof Samples = 63882, Incorrectly classified = 24781
Model 6...
Bonafide Samples = 7355, Incorrectly classified = 47
Spoof Samples = 63882, Incorrectly classified = 25911
Model 7...
Bonafide Samples = 7355, Incorrectly classified = 48
Spoof Samples = 63882, Incorrectly classified = 26070
Model 8...
Bonafide Samples = 7355, Incorrectly classifi

In [22]:
import pickle
with open("y_true_spectrogram.txt", "wb") as fp:   #Pickling
    pickle.dump(y_true, fp)

with open("y_pred_spectrogram.txt", "wb") as fp:   #Pickling
    pickle.dump(y_pred, fp)

In [23]:
with open("y_pred_spectrogram.txt", "rb") as fp:   # Unpickling
    b = pickle.load(fp)
print(len(b))
print(len(b[0]))

10
71237
